## Generating OOD samples

In [ ]:
from pathlib import Path
import os
from PIL import Image
import tensorflow as tf

In [ ]:
def add_gaussian_noise(image, mean=0.0, stddev=30.0):
    image = tf.cast(image, tf.float32)  # mantieni valori in [0,255]
    noise = tf.random.normal(shape=tf.shape(image), mean=mean, stddev=stddev, dtype=tf.float32)
    noisy_image = image + noise
    noisy_image = tf.clip_by_value(noisy_image, 0.0, 255.0)
    return noisy_image

def gaussian_kernel(size=11, sigma=5.0):
    x = tf.range(-size // 2 + 1, size // 2 + 1, dtype=tf.float32)
    x = tf.reshape(x, [-1, 1])
    y = tf.reshape(x, [1, -1])
    kernel = tf.exp(-(x**2 + y**2) / (2.0 * sigma**2))
    kernel /= tf.reduce_sum(kernel)
    kernel = tf.reshape(kernel, [size, size, 1, 1])
    return kernel

def add_gaussian_blur(image, kernel_size=11, sigma=5.0):
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, axis=0)  # Add batch dim
    kernel = gaussian_kernel(kernel_size, sigma)
    channels = tf.shape(image)[-1 ]# Repeat kernel for each channel (depthwise)
    kernel = tf.tile(kernel, [1, 1, channels, 1])
    blurred = tf.nn.depthwise_conv2d(image, kernel, strides=[1, 1, 1, 1], padding="VALID")
    return tf.squeeze(blurred, axis=0)

def process_ood(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = add_gaussian_blur(add_gaussian_noise(image))
    return tf.image.resize(image, [224, 224])

In [ ]:
root_dir = "/kaggle/input/camel-inference-dataset"
save_dir = "/kaggle/working/data"
os.makedirs(save_dir, exist_ok=True)

ood_list = []
keep_list = []

count_b = 0
count_e = 0
count_s = 0

for subtype in os.listdir(root_dir):  # biphasic, epithelioid
    subtype_path = os.path.join(root_dir, subtype)

    for patient_id in sorted(os.listdir(subtype_path)):  # sorted per avere ordine stabile
        patient_folder = os.path.join(subtype_path, patient_id)

        label = subtype.split("_")[1]
        
        if label == 'biphasic' and count_b < 2:
            target_subdir = "patches_ood"
            count_b += 1
            ood_list.append(patient_id)
        elif label == 'epithelioid' and count_e < 2:
            target_subdir = "patches_ood"
            count_e += 1
            ood_list.append(patient_id)
        elif label == 'sarcomatoid' and count_s < 2:
            target_subdir = "patches_ood"
            count_s += 1
            ood_list.append(patient_id)
        else:
            continue

        target_folder = os.path.join(save_dir, target_subdir, patient_id)
        os.makedirs(target_folder, exist_ok=True)

        for patch_file in os.listdir(patient_folder):
            if patch_file.endswith(".png"):
                patch_path = os.path.join(patient_folder, patch_file)

                if target_subdir == "patches_ood":
                    tensor = process_ood(patch_path)  # Definisci questa funzione
                    if tensor.dtype.is_floating:
                        tensor = tf.clip_by_value(tensor, 0.0, 255.0)
                        tensor = tf.cast(tensor, tf.uint8)
                    patch = Image.fromarray(tensor.numpy())
                else:
                    patch = Image.open(patch_path)

                patch.save(os.path.join(target_folder, patch_file))

print("OOD WSI:", ood_list)
print("Kept WSI:", keep_list)

In [ ]:
from pathlib import Path
import os
from PIL import Image
import tensorflow as tf

root_dir = "/kaggle/input/camel-dataset"
save_dir = "/kaggle/working/data"
os.makedirs(save_dir, exist_ok=True)

ood_list = []
keep_list = []

count_s = 0

for subtype in os.listdir(root_dir):  # biphasic, epithelioid
    subtype_path = os.path.join(root_dir, subtype)

    for patient_id in sorted(os.listdir(subtype_path)):  # sorted per avere ordine stabile
        patient_folder = os.path.join(subtype_path, patient_id)

        label = subtype.split("_")[1]

        if label == 'biphasic':
            continue
        elif label == 'epithelioid':
            continue
        elif label == 'sarcomatoid' and count_s < 2:
            target_subdir = "patches_ood"
            count_s += 1
            ood_list.append(patient_id)
        elif label == 'sarcomatoid' and count_s >= 2 and count < 4:
            target_subdir = subtype
            keep_list.append(patient_id)
        else:
            continue

        target_folder = os.path.join(save_dir, target_subdir, patient_id)
        os.makedirs(target_folder, exist_ok=True)

        for patch_file in os.listdir(patient_folder):
            if patch_file.endswith(".png"):
                patch_path = os.path.join(patient_folder, patch_file)

                if target_subdir == "patches_ood":
                    tensor = process_ood(patch_path)  # Definisci questa funzione
                    if tensor.dtype.is_floating:
                        tensor = tf.clip_by_value(tensor, 0.0, 255.0)
                        tensor = tf.cast(tensor, tf.uint8)
                    patch = Image.fromarray(tensor.numpy())
                else:
                    patch = Image.open(patch_path)

                patch.save(os.path.join(target_folder, patch_file))

print("OOD WSI:", ood_list)
print("Kept WSI:", keep_list)